In [1]:
import torch
import numpy as np
from numpy import asarray
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import permute
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
import pickle
import pandas as pd

In [2]:
#TODO module docstring, examples for functions

class CustomLabelEncoder:
    """
    Creates a mapping between string labels and integer class clabels for working with categorical data.
    
    
    Attributes
    ----------
    mapper:None dict
        None if mapper is not supplied or model is not fit.
        keys are unique string labels, values are integer class labels.
    """
    def __init__(self, mapper=None):
        """
        Initializes class instance.
        
        If the mapper dictionary is supplied here, then the model can be used without calling .fit().
        
        Parameters
        -----------
        mapper (optional): dict or None
            if mapper is None encoder will need to be fit to data before it can be used.
            If it is a dictionary mapping string labels to integer class labels, then this will be stored
            and the model can be used to transform data.
        """
        self.mapper = mapper
    
    def fit(self, str_labels, sorter=None):
        """
        Fits string labels to intiger indices with optional sorting.
        
        np.unique() is used to extract the unique values form labels. If 
        
        Parameters
        ----------
        str_labels: list-like
            list or array containing string labels
        
        sorter (optional): None or function
            key for calling sorted() on data to determine ordering of the numeric indices for each label.
            
        Attributes
        -----------
        mapper: dict
            dictionary mapping string labels to the sorted integer indices is stored after fitting.
        
        """
        sorted_unique = sorted(np.unique(str_labels), key=sorter)
        mapper = {label: i for i, label in enumerate(sorted_unique)}
        self.mapper = mapper    

    def transform(self, str_labels):
        """
        Maps string labels to integer labels.
        
        Parameters
        ----------
        str_labels: list-like
            list of string labels whose elements are in self.mapper
        
        Returns
        --------
        int_labels: array
            array of integer labels  corresponding to the string labels
        """
        assert self.mapper is not None, 'Encoder not fit yet!'
        
        int_labels = np.asarray([self.mapper[x] for x in str_labels], np.int)
        
        return int_labels
        
    def inverse_transform(self, int_labels):
        """
        Maps integer labels to original string labels.
        
        Parameters
        -----------
        int_labels: list-like
            list or array of integer class indices
        
        Returns
        ----------
        str_labels: array(str)
            array of string labels corresponding to intiger indices
        
        """
        assert self.mapper is not None, 'Encoder not fit yet!'
        
        reverse_mapper = {y:x for x,y in self.mapper.items()}
        
        str_labels = np.asarray([reverse_mapper[x] for x in int_labels])
        
        return str_labels
    
    @property
    def labels_ordered(self):
        """
        Returns an array containing the string labels in order of which they are stored.
        
        For example, if the label_encoder has the following encoding: {'a':1,'c':3,'b':2},
        then this will return array(['a','b','c'])
        """
        pass
    
    @labels_ordered.getter
    def labels_ordered(self):
        return self.inverse_transform(range(len(self.mapper)))

#### Loading Files

In [11]:
# load pre processed files
imgs_path = Path('/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_299_clipped_pred_polygon_225/')
assert imgs_path.is_dir()
files = sorted(imgs_path.glob('*.png'))

randomizer = np.random.RandomState(seed=99837)
randomizer.shuffle(files)

assert len(files) == int(str(imgs_path).split('_')[6]) # all files are found
print("First 10 files are: {}".format([x.name for x in files[:10]]))

First 10 files are: ['CFB184_ortho_rgb_mask_0276_12_preprocessed.png', 'CFB184_ortho_rgb_mask_0120_12_preprocessed.png', 'CFB184_ortho_rgb_mask_0022_12_preprocessed.png', 'CFB184_ortho_rgb_mask_0042_10_preprocessed.png', 'CFB184_ortho_rgb_mask_0257_10_preprocessed.png', 'CFB184_ortho_rgb_mask_0077_12_preprocessed.png', 'CFB184_ortho_rgb_mask_0021_12_preprocessed.png', 'CFB184_ortho_rgb_mask_0119_12_preprocessed.png', 'CFB184_ortho_rgb_mask_0086_10_preprocessed.png', 'CFB184_ortho_rgb_mask_0053_10_preprocessed.png']


#### Loading Images

In [12]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def alter_image(img_name):
    image = Image.open(img_name)
    image_tensor = normalize(to_tensor(image)).unsqueeze(0)
    image_tensor = image_tensor.reshape(1,3,299,299)
    return image_tensor

def load_images_as_tensors(paths):
    images = [alter_image(image) for image in paths]
    return images

image_tensors = load_images_as_tensors(files)
assert len(image_tensors) == int(str(imgs_path).split('_')[6])

RuntimeError: shape '[1, 3, 224, 224]' is invalid for input of size 157323

#### Extracting labels from filenames

In [21]:
def extract_labels(files): return [filename.stem.split('_')[5] for filename in files]
labels = extract_labels(files)
print('first 10 labels: {}'.format(labels[:10]))

first 10 labels: ['0', '12', '0', '8', '10', '10', '10', '12', '0', '12']


In [22]:
# change labels from number to species
update = {
    '0':'other',
    '4':'Fagus_sylvatica',
    '8':'deadwood',
    '10':'Abies_alba',
    '12':'Picea_abies'
}

updated_labels = (pd.Series(labels)).map(update)
species_labels = list(updated_labels)
labels = species_labels
print('first 10 labels: {}'.format(labels[:10]))

first 10 labels: ['other', 'Picea_abies', 'other', 'deadwood', 'Abies_alba', 'Abies_alba', 'Abies_alba', 'Picea_abies', 'other', 'Picea_abies']


#### Label encoding

Standardize encodings of labels to make analysis easier afterwards.

In [23]:
le = CustomLabelEncoder()
le.fit(labels, sorter=lambda x: x.upper())

labels_int = le.transform(labels[:10])
labels_str = le.inverse_transform(labels_int)

with open(Path('/home/richard/data/Schiefer/features/label_encoder_non_normalized.pickle'), 'wb') as f:
    pickle.dump(le, f)

print('label encodings: {}'.format(le.mapper))
print('first 10 integer labels: {}'.format(labels_int))
print('first 10 string labels: {}'.format(labels_str))

label encodings: {'Abies_alba': 0, 'deadwood': 1, 'Fagus_sylvatica': 2, 'other': 3, 'Picea_abies': 4}
first 10 integer labels: [3 4 3 1 0 0 0 4 3 4]
first 10 string labels: ['other' 'Picea_abies' 'other' 'deadwood' 'Abies_alba' 'Abies_alba'
 'Abies_alba' 'Picea_abies' 'other' 'Picea_abies']


/tmp/ipykernel_52833/3066462542.py:69: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  int_labels = np.asarray([self.mapper[x] for x in str_labels], np.int)


#### Feature Extraction

Load the ResNet152 model

In [24]:
# Load the pretrained model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
layer = model._modules.get('avgpool')
# Set model to evaluation mode
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [25]:
def get_feature_vector(image_tensor):
    # a dict to store the activations
    activation = {}
    def getActivation(name):
        # the hook signature
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    # register forward hooks on the layers of choice
    h1 = model.avgpool.register_forward_hook(getActivation('avgpool'))

    # forward pass -- getting the outputs
    out = model(image_tensor)

    # detach the hooks
    h1.remove()
    
    feature = torch.squeeze(activation['avgpool'])
    feature = torch.unsqueeze(feature,dim=0)
    return feature

# check
feature = get_feature_vector(image_tensors[0])
print(feature.shape)

torch.Size([1, 2048])


Get all features of the folder

In [26]:
def concat_tensors(features):
    fc = torch.cat(features)
    fc = fc.cpu().detach().numpy()
    return fc

def get_features(image_tensors):
    features = [get_feature_vector(tensor) for tensor in image_tensors]
    features = concat_tensors(features)
    return features

fc = get_features(image_tensors)
print(fc.shape)

(291, 2048)


Save features

In [27]:
results = {'filename': files,
           'features': fc,
           'labels': labels,
           'layer_name': 'fc'}

feature_dir = imgs_path.parent / 'features'
Path(feature_dir).mkdir(parents=True, exist_ok=True)
with open(feature_dir / 'Inception_v3_feature_std_224_normalized.pickle', 'wb') as f:
    pickle.dump(results, f)